In [ ]:
# default_exp utils

# Utils

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re
from functools import partial
from glob import glob
from typing import List

import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import xarray as xr
from dask import delayed
from dask_image.imread import imread
from matplotlib import pyplot as plt
from scipy.stats import mode
from skimage import measure, segmentation
import pyclesperanto_prototype as cle

In [ ]:
#export
def clean_img_names(img_path_glob, img_name_regex: str):
    return [
        re.findall(img_name_regex, os.path.basename(fn))[0]
        for fn in sorted(glob(img_path_glob))
    ]